<a href="https://colab.research.google.com/github/Sanarazaaa/ADHD-Game/blob/main/ADHD_Game_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
%%html
<!DOCTYPE html>
<html>

<head>
    <script src="https://d3js.org/d3.v7.min.js"></script>
</head>

<body>
    <h2>🎮 Circle Pop Challenge</h2>
    <button id="startBtn" onclick="startGame()">Start Game</button>
    <p>Pop the bubbles! Big Circle = +5, Small Circle = +20. Score: <span id="score">0</span></p>
    <svg width="800" height="500" style="border:2px solid black; background:linear-gradient(to bottom, #FFFAF0, #87CEEB); display:none;"></svg>

    <audio id="popSound" src="https://www.soundjay.com/button/sounds/button-3.mp3"></audio>

    <script>
        const svg = d3.select("svg");
        const popSound = document.getElementById("popSound");
        let score = 0;
        let bubbleSpeed = 5000; // Initial bubble lifespan
        let missedSmallCircleTime = 0;
        let gameInterval, timerInterval;

        // Update score display
        function updateScore() {
            d3.select("#score").text(score);
        }

        // Create circles (Big and Small)
        function createCircle() {
            const x = Math.random() * 750; // Random X position
            const y = Math.random() * 450; // Random Y position

            // 30% chance for small circle, 70% for big circle
            const isSmall = Math.random() < 0.3;
            const size = isSmall ? Math.random() * 20 + 10 : Math.random() * 50 + 40;
            const points = isSmall ? 20 : 5; // Higher points for small circles

            const circle = svg.append("circle")
                .attr("cx", x)
                .attr("cy", y)
                .attr("r", size)
                .attr("fill", d3.interpolateRainbow(Math.random()))
                .style("cursor", "pointer")
                .on("click", function () {
                    d3.select(this).remove();
                    score += points;
                    popSound.play();
                    updateScore();
                    increaseDifficulty();

                    // Reset the small circle timer if a small circle is popped
                    if (isSmall) missedSmallCircleTime = 0;
                });

            // Circle fades away after time
            circle.transition()
                .duration(bubbleSpeed)
                .style("opacity", 0)
                .remove();
        }

        // Increase difficulty as score increases
        function increaseDifficulty() {
            if (score % 50 === 0 && bubbleSpeed > 1500) {
                bubbleSpeed -= 300; // Bubbles fade faster
            }
        }

        // Timer to track if a small circle is popped
        function checkSmallCircleTimer() {
            missedSmallCircleTime += 1;
            if (missedSmallCircleTime >= 120) { // 2 minutes (120 seconds)
                endGame();
            }
        }

        // Start the game
        function startGame() {
            // Reset values
            score = 0;
            bubbleSpeed = 5000;
            missedSmallCircleTime = 0;

            updateScore();
            svg.style("display", "block");
            document.getElementById("startBtn").style.display = "none";

            // Generate circles and start timer
            gameInterval = setInterval(createCircle, 800);
            timerInterval = setInterval(checkSmallCircleTimer, 1000);
        }

        // End the game
        function endGame() {
            clearInterval(gameInterval);
            clearInterval(timerInterval);
            svg.style("display", "none");

            // Send score to Python for saving to Google Sheets
            google.colab.kernel.invokeFunction('save_score_to_sheet', [score]);

            alert(`Game Over! Your final score is: ${score}`);
            document.getElementById("startBtn").style.display = "block";
        }

    </script>
</body>

</html>

In [24]:
import gspread
from google.colab import auth, output

# Authenticate access to your Google Sheets
auth.authenticate_user()

# Connect to Google Sheets
from google.auth import default
credentials, _ = default()
gc = gspread.authorize(credentials)

# Spreadsheet URL
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1D1M2Gn0ImVd1-SjCkP03s2r2dtpQAyPElyRHLZ0YIXs/edit#gid=0'
spreadsheet = gc.open_by_url(spreadsheet_url)
worksheet = spreadsheet.sheet1 # Get the first worksheet

# Function to save the score to Google Sheets
def save_score_to_sheet(score):
  worksheet.append_row([score])  # Append the score to a new row
  print(f"Score {score} saved to Google Sheets!")

# Register the function for Javascript to call
output.register_callback('save_score_to_sheet', save_score_to_sheet)